In [25]:
import pandas as pd
import numpy as np
import scipy.sparse

In [26]:
df_1 = pd.read_csv('../data/fare_amount_src_dst_t_1.csv')
df_5 = pd.read_csv('../data/fare_amount_src_dst_t_5.csv')
df_10 = pd.read_csv('../data/fare_amount_src_dst_t_10.csv')
df_15 = pd.read_csv('../data/fare_amount_src_dst_t_15.csv')

In [27]:
df_1.head()

,pickup_datetime_index,pickup_datetime_interval,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
0,0,00:00:00,2.0,2.0,15.0,15.0,1
1,0,00:00:00,3.0,69.0,24.5,24.5,1
2,0,00:00:00,4.0,4.0,3.5,3.5,1
3,0,00:00:00,4.0,33.0,19.5,19.5,2
4,0,00:00:00,4.0,41.0,22.0,22.0,1


In [28]:
#create Marcov Chain Transition matrix with counts

def create_MC_mtx(df):
    # Marcov Chain Transition matrix with counts
    zone_num = 263
    mc_mtx = np.zeros((int(max(df['pickup_datetime_index']))+1, zone_num, zone_num))
    for i in range(len(df)):
        t = df['pickup_datetime_index'][i]
        pickup = int(df['pickup_taxizone_id'][i]) -1
        dropoff = int(df['dropoff_taxizone_id'][i]) -1
        mc_mtx[t, pickup, dropoff] = df['count'][i]   

    # convert counts to probability
    mc_mtx_prob = np.zeros((int(max(df['pickup_datetime_index']))+1, zone_num, zone_num))
    for i in range(mc_mtx.shape[0]):
        mc_mtx_prob[i] = mc_mtx[i]/np.sum(mc_mtx[i], axis = 1).reshape(zone_num, 1)
    mc_mtx_prob = np.nan_to_num(mc_mtx_prob)
    
    return mc_mtx_prob

In [19]:
mc_mtx_prob_1 = create_MC_mtx(df_1)
mc_mtx_prob_5 = create_MC_mtx(df_5)
mc_mtx_prob_10 = create_MC_mtx(df_10)
mc_mtx_prob_15 = create_MC_mtx(df_15)

C:\Users\stell\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [30]:
#check
df_15.loc[((df['pickup_taxizone_id'] == 217) & (df['pickup_datetime_index'] ==33)),:]

,pickup_datetime_index,pickup_datetime_interval,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
266288,33,08:15:00,217.0,61.0,9.0,9.0,1
266289,33,08:15:00,217.0,80.0,4.0,4.0,1
266290,33,08:15:00,217.0,97.0,11.0,11.0,1
266291,33,08:15:00,217.0,107.0,13.0,13.0,1


In [31]:
mc_mtx_prob_15[33,216,:]

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.

In [52]:
# saving reshaped array to file. 
mc_mtx_prob_1_reshaped = mc_mtx_prob_1.reshape(mc_mtx_prob_1.shape[0], -1) 
mc_mtx_prob_1_compressed = scipy.sparse.csc_matrix(np.array(mc_mtx_prob_1_reshaped))
scipy.sparse.save_npz('../data/mc_mtx_1.npz', mc_mtx_prob_1_compressed)

mc_mtx_prob_5_reshaped = mc_mtx_prob_5.reshape(mc_mtx_prob_5.shape[0], -1)
mc_mtx_prob_5_compressed = scipy.sparse.csc_matrix(np.array(mc_mtx_prob_5_reshaped))
scipy.sparse.save_npz('../data/mc_mtx_5.npz', mc_mtx_prob_5_compressed)

mc_mtx_prob_10_reshaped = mc_mtx_prob_10.reshape(mc_mtx_prob_10.shape[0], -1) 
mc_mtx_prob_10_compressed = scipy.sparse.csc_matrix(np.array(mc_mtx_prob_10_reshaped))
scipy.sparse.save_npz('../data/mc_mtx_10.npz', mc_mtx_prob_10_compressed)

mc_mtx_prob_15_reshaped = mc_mtx_prob_15.reshape(mc_mtx_prob_15.shape[0], -1)
mc_mtx_prob_15_compressed = scipy.sparse.csc_matrix(np.array(mc_mtx_prob_15_reshaped))
scipy.sparse.save_npz('../data/mc_mtx_15.npz', mc_mtx_prob_15_compressed)

## Try Functions

In [53]:
from estimations import Estimator

In [60]:
est = Estimator(delta_t=10, dir_path='../data/')

In [63]:
est.generate_request(217, 143)

256